In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# -*- coding: utf-8 -*-
"""chexpert_noisfy.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Up66s8R2xK4yRqBX0sGYIZpyZ1ncOnhd
"""

import os
import json
import pickle
import datetime
import threading
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import CSVLogger

import sys
sys.path.append('/content/gdrive/My Drive/chexpert_semantic_noise/noisy_data/baseline')
from tf_chexpert_utilities import *

In [6]:
def get_negative_cases():
  return ['Enlarged Cardiomediastinum', 'Cardiomegaly', 
          'Lung Opacity', 'Lung Lesion','Edema',
          'Consolidation', 'Pneumonia','Atelectasis',
          'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
          'Fracture', 'Support Devices']


def get_value_dict(noise):
  folder = f'./network_training_predictions/torch_baseline/{noise}'

  NEGATIVE_CASES = ['Enlarged Cardiomediastinum', 
                    'Cardiomegaly', 
                    'Lung Opacity', 
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture',
                    'Support Devices']

  m_list = ['loss', 
            'acc', 
            'real_loss', 
            'real_acc', 
            'recall', 
            'precision', 
            'f_one_score',
            'positive_acc',
            'negative_acc',
            'positive_real_acc',
            'negative_real_acc']

  # finish metric keeper

  train_m_list = []
  val_m_list = []

  for m in m_list:
    train_m_list.append(f'train_{m}'.lower())
    val_m_list.append(f'val_{m}'.lower())
  for n in NEGATIVE_CASES:
    k = n.lower().replace(' ', '_').lower()
    train_m_list.append(f'train_{k}_acc')
    train_m_list.append(f'train_{k}_real_acc')
    val_m_list.append(f'val_{k}_acc')
    val_m_list.append(f'val_{k}_real_acc')

  print(train_m_list)
  print(val_m_list)

  value_dict = {}
  for i in ['train', 'val']:
    value_dict[i] = {}
    m_list = train_m_list
    if i == 'val':
      m_list = val_m_list
    for j in m_list:
      fname = f'{folder}/{j}_e'
      print(f'getting {fname}')
      value_dict[i][j] = unpickle(fname)

  return value_dict


def plot_each_case(value_name, train_case_values, val_case_values, noise):
  plt.style.use('seaborn-darkgrid')
  #get fig
  fig, ax = plt.subplots(figsize=(10, 10))
  # title
  ax.set_title(f'Result:{value_name}({noise}% noise)', color='C0')
  # plot values
  ax.plot(np.arange(len(train_case_values)), train_case_values, 'midnightblue', label=f'train {value_name}')
  ax.plot(np.arange(len(val_case_values)), val_case_values, 'seagreen', label=f'val {value_name}')
  ax.set_xlabel('epochs')
  ax.set_ylabel(value_name)
  ax.legend()
  fig.savefig(f'{value_name}_{noise}.jpg')

def plot_positive_and_negative(value_name, train_positive_values, train_negative_values, val_positive_values, val_negative_values, noise):
  plt.style.use('seaborn-darkgrid')
  #get fig
  fig, ax = plt.subplots(figsize=(10, 10))
  # title
  ax.set_title(f'Result:{value_name}({noise}% noise)', color='C0')
  # plot values
  ax.plot(np.arange(len(train_positive_values)), train_positive_values, 'midnightblue', label=f'train positive {value_name}')
  ax.plot(np.arange(len(train_negative_values)), train_negative_values, 'seagreen', label=f'train negative {value_name}')
  ax.plot(np.arange(len(val_positive_values)), val_positive_values, 'darkred', label=f'val positive {value_name}')
  ax.plot(np.arange(len(val_negative_values)), val_negative_values, 'darkorange', label=f'val negative {value_name}')
  # labels
  ax.set_xlabel('epochs')
  ax.set_ylabel(value_name)
  ax.legend()
  fig.savefig(f'positive_negative_{value_name}_{noise}.jpg')

def plot_values(train_values, val_values, value_name, noise):
  plt.style.use('seaborn-darkgrid')
  #plt.xlabel('epochs')
  #plt.ylabel(value_name)

  fig, ax = plt.subplots(figsize=(10, 10))
  ax.set_title(f'Result:{value_name}({noise}% noise)', color='C0')
  ax.plot(np.arange(len(train_values)), train_values, 'C9', label='train')
  ax.plot(np.arange(len(val_values)), val_values, 'C3', label='val')
  ax.set_xlabel('epochs')
  ax.set_ylabel(value_name)
  #fig.xlabel('epochs')
  #fig.ylabel(value_name)
  ax.legend()
  fig.savefig(f'{value_name}_{noise}.jpg')

def g_plot_cases(value_dict, noise):
  for n in get_negative_cases():
    value_name = n + ' ACC'
    k = n.lower().replace(' ', '_').lower()
    t = f'train_{k}_acc'
    v = f'val_{k}_acc'

    train_case_values = value_dict['train'][t]
    #print(f'train_case_values for [{t}]: {train_case_values}')
    val_case_values = value_dict['val'][v]
    plot_each_case(value_name, train_case_values, val_case_values, noise)

def g_plot_positive_and_negative(value_dict, noise):
  value_name = 'acc'
  plot_positive_and_negative(value_name, value_dict['train']['train_positive_acc'], value_dict['train']['train_negative_acc'], value_dict['val']['val_positive_acc'], value_dict['val']['val_negative_acc'], noise)

def g_plot_basics(value_dict, noise):
  for j in ['loss', 'real_loss', 'acc', 'real_acc', 'positive_acc', 'negative_acc' ,'positive_real_acc', 'negative_real_acc']:
    value_name = j
    plot_values(value_dict['train'][f'train_{value_name}'], value_dict['val'][f'val_{value_name}'], value_name, noise=noise)